In [ ]:
img = tf.io.read_file(path)
img = tf.image.decode_jpeg(img,channels=3)
img = tf.image.resize(image,(200,200))


## Model
* example 
* shape(64,2048)

In [ ]:
class BahdanauAttention(tf.keras.Model):
    def __init__(self,units):
        super(BahdanauAttention,self)__init__()
        self.W1 = tf.keras.layers.Dense(uints)
        self.W2 = tf.keras.layers.Dense(uints)
        self.V = tf.keras.layers.Dense(1)
        
    def call(self,feature,hidden):
        # feature shape(batch_size,timestep,feature_size)
        # hidden shape(batch_size,hidden_size)
        # hidden_with_time_axis(batch_size,1,hidden_size)
        hidden_with_time_axis = tf.expand_dims(hidden,1)
        
        #score (batch,timestep,units) timestep=64
        score = tf.nn.tanh(self.W1(feature) + self.W2(hidden_with_time_axis))
        #score (batch,timestep,1)
        score = self.V(score)
        attention_weights = tf.nn.softmax(score,axis=1)
        context_vector = attention_weights * feature
        context_vector = tf.reduce_sum(context_vector,axis=1)
        
        return context_vector, attention_weights
        
        

In [ ]:
class CNN_Encoder(tf.keras.Model):
    def __init__(self,embedding_dim):
        super(CNN_Encoder,self).__init__()
        self.fc = tf.keras.layers.Dense(embedding_dim)
    def call(self,x)
        x = self.fc(x)
        x = tf.nn.relu(x)
        return x

In [ ]:
class RNN_Decoder(tf.keras.Model):
    def __init__(self,embedding_dim,units,vocab_size):
        super(RNN_Decoder,self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size,embedding_dim)
        self.gru  = tf.keras.layers.GRU(self.units,return_sequences=True,
                                        return_state=True,
                                        recurrent_initializer='glorot_uniform')
        self.fc1 = tf.keras.Dense(self.units)
        self.fc2 = tf.keras.Dense(vocab_size)
        self.attention = BahdanauAttention(self.units)
        
    def call(self,x,features,hidden):
        context_vector, attention_weights = self.attention(features,hidden)
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector,axis=1),x],axis=-1)
        gru_output, gru_state = self.gru(x)
        
        # shape == (batch_size, max_length, hidden_size)
        x = self.fc1(gru_output)
        # x shape == (batch_size * max_length, hidden_size)
        x = tf.reshape(x,(-1,x.shape[2]))
        # output shape == (batch_size * max_length, vocab)
        x = self.fc2(x)
        return x, gru_state,attention_weights
        
    def reset_state(self, batch_size):
        return tf.zeros((batch_size, self.units))      

In [ ]:
encoder = CNN_Encoder(embedding_dim)
decoder = RNN_Decoder(embedding_dim, units, vocab_size)

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real,pred):
    mask = tf.math.logical_not(tf.math.equal(real,0))
    loss_ = loss_object(real,pred)
    mask = tf.cast(mask,dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

In [ ]:
# adding this in a separate cell because if you run the training cell
# many times, the loss_plot array will be reset
loss_plot = []

In [ ]:
@tf.function
def train_step(image_tensor,target):
    loss = 0
    
    ## initial hidden state
    hidden = decoder.reset_state(target.shape[0])
    # shape (batch,1)
    dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * target.shape[0],1)
    
    with tf.GradientTape() as tape:
        features = encoder(image_tensor)
        for i in range(1,target.shape[1]):
            predictions,hidden,_ = decoder(dec_input,features,hidden)
            loss += loss_function(target[:,i],predictions)
            dec_input = tf.expand_dims(target[:,i],1)
        
    total_loss = (loss/ int(target.shape(1)))
    
    trainable_variables = encoder.trainable_variables =  decoder.trainable_variables
    gradients = tape.gradient(loss,trainable_variables)
    optimizer.apply_gradients(zip(gradients, trainable_variables))
    return loss,total_loss

In [ ]:
EPOCHS = 20
for epoch in range(start_epoch,EPOCHS):
    start = time.time()
    total_loss = 0
    
    for (batch,(image_tensor,target) in enumerate(dataset)):
        batch_loss, t_loss = train_step(image_tensor,target)
        total_loss += t_loss
        
        if batch % 100 == 0:
            print ('Epoch {} Batch {} Loss {:.4f}'.format(
              epoch + 1, batch, batch_loss.numpy() / int(target.shape[1])))
    # storing the epoch end loss value to plot later
    loss_plot.append(total_loss / num_steps)

    if epoch % 5 == 0:
      ckpt_manager.save()

    print ('Epoch {} Loss {:.6f}'.format(epoch + 1,
                                         total_loss/num_steps))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))